In [1]:
import os, re, sys, utilities
spark_home = os.environ.get('SPARK_HOME', None)

In [2]:
code = utilities.getSparkContext()
exec(code)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2.3.1.4.26-3
      /_/

Using Python version 3.6.8 (default, Dec 30 2018 01:22:34)
SparkSession available as 'spark'.


In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, HiveContext
import matplotlib
from pyspark.sql.types import DoubleType

In [4]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql.functions import col, countDistinct, isnan, when, count, to_date, from_unixtime, unix_timestamp

### Read in parquet file from HDFS

In [ ]:
#example of reading data from a table instead of parquet file
MD_crashes_tbl = spark.sql('select acc_date, collision_type_code, fix_obj_code, harm_event_code1, harm_event_code2, light_code, report_no, surf_cond_code, weather_code, damage_code, movement_code, vin_no, veh_year, veh_make, commercial_flag, towed_away_flag, going_direction_code, driverless_flag, fire_flag, parked_flag, speed_limit, hit_and_run_flag, vehicle_id, area_damaged_code_imp1, area_damaged_code1, area_damaged_code2, area_damaged_code3, area_damaged_code_main from di.md_crashes where veh_make in ("FFORD", "FORD", "FORD`", "FORF", "FORK", "FORM", "FORN", "FORRD", "FORS", "FPRD", "FRD", "LIN", "LINC", "LINCOLIN", "LINCOLN", "LINCON", "MECRURY", "MECURY")')

In [5]:
# using SQLContext to read parquet file
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(SparkContext)

# to read parquet file
df = spark.read.parquet('hdfs://hdp2cluster/user/lguerra5/prototype_model_output_new/*')

In [6]:
df.show(5)

+---------------+-----------------+-----------+------------------+-----------------+------------------+
|next_oil_change|next_oil_odometer|num_samples|         r_squared|              vin|weighted_r_squared|
+---------------+-----------------+-----------+------------------+-----------------+------------------+
|     2019-12-20|             8510|        477|0.9895600519860865|1FA6P8CF0K5146513|0.9946460096459021|
|     2020-05-09|            16405|        401| 0.928696149018734|1FA6P8CF0K5188079|0.9652292987437842|
|     2020-02-14|            33131|       1500|0.9341943683080315|1FA6P8CF1J5120310|0.9904339587907416|
|     2020-10-31|             4964|        264|0.9669387742307612|1FA6P8CF1K5103492|0.9954691867737939|
|     2020-07-22|            11796|        903|0.9979681852967399|1FA6P8CF1K5137982|0.9976790989879895|
+---------------+-----------------+-----------+------------------+-----------------+------------------+
only showing top 5 rows



In [25]:
from datetime import timedelta

In [26]:
# compute original bins function
# note - when its an int or float use 'and' instead of '&' for if statements
def GetBins(r):
    # Initialize Pandas DataFrame
    df = pd.DataFrame({'next_oil_change': [pd.to_datetime(str(r[0]), format='%Y-%m-%d').date()], 'next_oil_odometer': [r[1]], 'num_samples': [r[2]], 'r_squared': [r[3]], 'vin': [r[4]], 'weighted_r_squared': [r[5]]})
    
    next_oc= pd.to_datetime(str(r[0]), format='%Y-%m-%d').date()
    
    #compute original bin value (r2)        
    if (r[3] >= 0.5 and r[2] >= 1000):
        if (next_oc <= (pd.to_datetime('now').date()+timedelta(days=60))):
            #df["orig_bin"] = 1
            og_bin = 1
        else:
            #df["orig_bin"] = 2
            og_bin = 2
    elif (r[3] >= 0.5 and r[2] >= 400):
        #df["orig_bin"] = 3
        og_bin = 3
    else:
        #df["orig_bin"] = 4
        og_bin = 4 
            
    return {'next_oil_change': r[0], 'next_oil_odometer': r[1], 'num_samples': r[2], 'r_squared': r[3], 
            'vin': r[4], 'weighted_r_squared': r[5],'orig_bin': og_bin}

In [27]:
rdd_bins = df.rdd.map(lambda r: GetBins(r))

In [28]:
df_bins = rdd_bins.toDF()

/usr/hdp/current/spark2-client/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [30]:
df_bins.show(5)

+-------+---------------+-----------------+-----------+--------+------------------+-----------------+------------------+
|new_bin|next_oil_change|next_oil_odometer|num_samples|orig_bin|         r_squared|              vin|weighted_r_squared|
+-------+---------------+-----------------+-----------+--------+------------------+-----------------+------------------+
|      3|     2019-12-20|             8510|        477|       3|0.9895600519860865|1FA6P8CF0K5146513|0.9946460096459021|
|      3|     2020-05-09|            16405|        401|       3| 0.928696149018734|1FA6P8CF0K5188079|0.9652292987437842|
|      2|     2020-02-14|            33131|       1500|       2|0.9341943683080315|1FA6P8CF1J5120310|0.9904339587907416|
|      4|     2020-10-31|             4964|        264|       4|0.9669387742307612|1FA6P8CF1K5103492|0.9954691867737939|
|      3|     2020-07-22|            11796|        903|       3|0.9979681852967399|1FA6P8CF1K5137982|0.9976790989879895|
+-------+---------------+-------

In [29]:
df_o = df_bins.toPandas()

In [33]:
#df_o.head()